In [1]:
from __future__ import division
from scipy import stats
from scipy.stats.stats import pearsonr, spearmanr
import scikits.bootstrap as boot
from scipy.stats import ttest_rel
import itertools
import pyvttbl
from pyvttbl import DataFrame as DF
from pyvttbl.stats import Anova, Anova1way

colors=["#0000B8",'#005CE6',"#89a0ff","#990000" ,'#FF1919', "#ea8989"]
clist=[blues[-4], blues[-3], reds[-4], reds[-2]]

In [28]:
wtc_ffa.loc[:, '1':'12'].max(axis=1)

0      0.2540
1      0.2668
2      0.2664
3      0.3539
4      0.5399
5      0.3666
6      0.1057
7      0.1820
8      0.0888
9      0.5832
10     0.4752
11     0.7173
12     1.5460
13     0.9781
14     0.9146
        ...  
156    0.4805
157    0.5604
158    0.3717
159    0.0966
160    0.0767
161    0.0504
162    0.0327
163    0.0054
164    0.0839
165    0.2242
166    0.1293
167    0.1969
168    0.0026
169    0.1681
170    0.2095
dtype: float64

In [23]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/idxPK')

#WIN Stim Phase Peaks coded by Cue in FFA
wpk_ffa=pd.read_csv("ffa_rho_df.csv", index_col='idx')

#just grab the stim phase
ffa_cstim = wpk_ffa[['hcF', 'ncF', 'fcF', 'hcH', 'ncH', 'fcH']]

###Wide-Format 
* each condition as a column
* each row is a subject
* values in matrix are observed values of DV 
    
###Long-Format
* columns are SubjectID and names of  IV(s), DV(s)
* SubjectID contain ID name for every combination if IV factors 
    * i.e. Subject 1 occurs 8 times if Design is  IV1(2 levels) X IV2(4 levels)

* **rows in IV column**: name of IV levels are levels
* **rows in DV column**: observed value for conditon name in that row

In [98]:
# Wide
ffa_cstim.head(3)

,hcF,ncF,fcF,hcH,ncH,fcH
idx,,,,,,
8,0.3539,0.5399,0.3666,0.1057,0.1820,0.0888
10,1.5460,0.9781,0.9146,0.4596,0.1573,0.4097
11,0.2857,0.1608,0.2483,0.0158,0.2287,0.2283


In order to run a rep. measures anova we need the data to be in a long-format style, where we have a column for each of our IVs (all rows are levels of IV) and DVs (all rows the value observed for the condition in the IV column)

In [3]:
# Stack to multi-index with all column names nested under each subject idx
# then reset the index (single level), name the DV peak
ffa_cs_long = ffa_cstim.stack().reset_index(name='peak')
# old column names now rows (N times) and need a name 
# (defaults to "level_1" after reset)
ffa_cs_long.rename(columns={'level_1':'cstim'}, inplace=True)
ffa_cs_long.head(12)

,idx,cstim,peak
0,8,hcF,0.3539
1,8,ncF,0.5399
2,8,fcF,0.3666
3,8,hcH,0.1057
4,8,ncH,0.1820
5,8,fcH,0.0888
6,10,hcF,1.5460
7,10,ncF,0.9781
8,10,fcF,0.9146
9,10,hcH,0.4596


## 2-way repeated measures ANOVA 
we need to split up cstim so that we have separate columns for cue and stimulus for each trial 

#### Current Format:
* **cstim**: [hcF, ncF, fcF, hcH, ncH, fcH] x N

#### Transformed: 
* **stim**: [F, H] x N x 3 (cue levels)
* **cue**: [hc, nc, fc] x N x 2 (stim levels)

In [109]:
ffa_cs_long['cue']=[cs[:2] for cs in ffa_cs_long['cstim']]
ffa_cs_long['stim']=[cs[-1] for cs in ffa_cs_long['cstim']]

In [111]:
ffa_cs_long.head(12)

,idx,cstim,peak,cue,stim
0,8,hcF,0.3539,hc,F
1,8,ncF,0.5399,nc,F
2,8,fcF,0.3666,fc,F
3,8,hcH,0.1057,hc,H
4,8,ncH,0.1820,nc,H
5,8,fcH,0.0888,fc,H
6,10,hcF,1.5460,hc,F
7,10,ncF,0.9781,nc,F
8,10,fcF,0.9146,fc,F
9,10,hcH,0.4596,hc,H


##Run Repeated Measures ANOVA with pyvttbl

In [117]:
import pyvttbl
from pyvttbl import DataFrame as DF
from pyvttbl.stats import Anova

FFA_CStim = DF(ffa_cs_long)
FFA_CStim_RMA=Anova()
FFA_CStim_RMA.run(FFA_CStim, 'peak', wfactors=['cue', 'stim'], sub='idx')

os.chdir('/Users/kyle/Dropbox/PFH/iPFH/PK_ANOVA/')
# Write output to a file
with open('FFA_CStim_RMA.txt','wb') as f:
        f.write(str(FFA_CStim_RMA))

# Display output inline
print(FFA_CStim_RMA)

peak ~ cue * stim

TESTS OF WITHIN SUBJECTS EFFECTS

Measure: peak
  Source                           Type III    eps      df      MS       F        Sig.      et2_G   Obs.    SE     95% CI   lambda    Obs.  
                                      SS                                                                                              Power 
cue           Sphericity Assumed      0.046       -        2   0.023    0.477       0.624   0.004     38   0.037    0.072     1.008   0.126 
              Greenhouse-Geisser      0.046   0.899    1.798   0.025    0.477       0.605   0.004     38   0.037    0.072     1.008   0.122 
              Huynh-Feldt             0.046   0.899    1.798   0.025    0.477       0.605   0.004     38   0.037    0.072     1.008   0.122 
              Box                     0.046   0.500        1   0.046    0.477       0.498   0.004     38   0.037    0.072     1.008   0.103 
---------------------------------------------------------------------------------------

## Now the same test for PPA CStim peaks

In [9]:
wpk_ppa

,hc,nc,fc,hcF,ncF,fcF,hcH,ncH,fcH,fc-fcH,fc-fcF,hc-hcH,hc-hcF
idx,,,,,,,,,,,,,
8,0.1061,0.0548,0.0384,0.0568,0.1003,0.0955,0.2313,0.2968,0.3283,0.2898,0.0571,0.1252,-0.0493
10,0.0198,0.0043,0.0150,0.1524,0.0766,0.0665,0.0470,0.0938,0.1999,0.1849,0.0515,0.0272,0.1326
11,0.0751,0.1020,0.0798,0.0942,0.1217,0.0912,0.1573,0.1598,0.2435,0.1636,0.0113,0.0822,0.0191
13,0.0964,0.0881,0.1392,0.0412,0.0577,0.0510,0.1665,0.1622,0.1751,0.0359,-0.0882,0.0701,-0.0553
15,0.1677,0.1821,0.1698,0.1027,0.1488,0.2071,0.4065,0.3562,0.5679,0.3981,0.0372,0.2388,-0.0651
16,0.0874,0.0355,0.1298,0.1811,0.1834,0.1069,0.3261,0.4447,0.4759,0.3461,-0.0230,0.2387,0.0937
17,0.0981,0.0500,0.0562,0.1573,0.1107,0.1303,0.1079,0.1529,0.2998,0.2435,0.0740,0.0099,0.0592
18,0.0556,0.0837,0.0390,0.0620,-0.0129,0.0951,0.2963,0.2471,0.2512,0.2122,0.0562,0.2407,0.0064
19,0.1126,0.1388,0.0830,0.1524,0.0852,0.1367,0.1945,0.2835,0.3750,0.2920,0.0537,0.0819,0.0397


In [12]:
wlo_ppa.hcF

idx
8    -0.0435
10   -0.0576
11   -0.0613
13   -0.0266
15   -0.1800
16   -0.0544
17   -0.0208
18   -0.1646
19   -0.1983
20   -0.0633
21   -0.1700
22   -0.0251
23   -0.0824
24   -0.0815
26   -0.0688
27       NaN
29       NaN
30       NaN
31       NaN
Name: hcF, dtype: float64

In [11]:
ttest_rel(wpk_ppa.fcH, wpk_ppa.hcH)#max(axis=1)
ttest_rel(wlo_ppa.hcF, wlo_ppa.fcF)

(nan, nan)

In [4]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/idxPK/')
wpk_ppa=pd.read_csv("ppa_rho_df.csv", index_col='idx')
ppa_cstim = wpk_ppa[['hcF', 'ncF', 'fcF', 'hcH', 'ncH', 'fcH']]

ppa_cs_long = ppa_cstim.stack().reset_index(name='peak').rename(columns={'level_1':'cstim'})

ppa_cs_long['cue']=[cs[:2] for cs in ppa_cs_long['cstim']]
ppa_cs_long['stim']=[cs[-1] for cs in ppa_cs_long['cstim']]

PPA_CStim = DF(ppa_cs_long)
PPA_CStim_RMA=Anova()
PPA_CStim_RMA.run(PPA_CStim, 'peak', wfactors=['cue', 'stim'], sub='idx')

#os.chdir('/Users/kyle/Dropbox/PFH/iPFH/PK_ANOVA/')
# Write output to a file
#with open('PPA_CStim_RMA.txt','wb') as f:
#        f.write(str(PPA_CStim_RMA))

# Display output inline
print(PPA_CStim_RMA)

peak ~ cue * stim

TESTS OF WITHIN SUBJECTS EFFECTS

Measure: peak
  Source                           Type III    eps      df      MS       F        Sig.      et2_G   Obs.    SE     95% CI   lambda    Obs.  
                                      SS                                                                                              Power 
cue           Sphericity Assumed      0.018       -        2   0.009    2.267       0.118   0.016     38   0.011    0.021     4.785   0.452 
              Greenhouse-Geisser      0.018   0.860    1.721   0.010    2.267       0.127   0.016     38   0.011    0.021     4.785   0.415 
              Huynh-Feldt             0.018   0.860    1.721   0.010    2.267       0.127   0.016     38   0.011    0.021     4.785   0.415 
              Box                     0.018   0.500        1   0.018    2.267       0.150   0.016     38   0.011    0.021     4.785   0.310 
---------------------------------------------------------------------------------------

In [ ]:
ttest_rel()

In [29]:
wpk_ppa=pd.read_csv("ppa_rho_df.csv", index_col='idx')

condlist = ['hc', 'nc', 'fc']
ppacues = wpk_ppa[condlist]
ppa_cues_long = ppacues.stack().reset_index(name='peak').rename(columns={'level_1':'cue'})
PPA_CUES = DF(ppa_cues_long)
PPA_CUE_RMA=Anova()
PPA_CUE_RMA.run(PPA_CUES, 'peak', wfactors=['cue'], sub='idx')

os.chdir('/Users/kyle/Dropbox/PFH/iPFH/PK_ANOVA/')
# Write output to a file
with open('PPA_CUE_RMA.txt','wb') as f:
      f.write("House Regions, Cue Peak Comparisons:\n\n")
      f.write("House Cue > Face Cue:\nt(18) = %.4f, p=%.4f\n\n" % ttest_rel(wpk_ppa['hc'], wpk_ppa['fc']))
      f.write("House Cue > Neutral Cue:\nt(18) = %.4f, p=%.4f\n\n\n\n" % ttest_rel(wpk_ppa['hc'], wpk_ppa['nc']))
      f.write("\n\n")
      f.write(str(PPA_CUE_RMA))

print "House Regions, Cue Peak Comparisons"
print "House Cue > Face Cue: t(18) = %.4f, p=%.4f" % ttest_rel(wpk_ppa['hc'], wpk_ppa['fc'])
print "House Cue > Neutral Cue: t(18) = %.4f, p=%.4f" % ttest_rel(wpk_ppa['hc'], wpk_ppa['nc'])
# Display output inline
print(PPA_CUE_RMA)

House Regions, Cue Peak Comparisons
House Cue > Face Cue: t(18) = -0.0419, p=0.9670
House Cue > Neutral Cue: t(18) = 0.1378, p=0.8919
peak ~ cue

TESTS OF WITHIN SUBJECTS EFFECTS

Measure: peak
  Source                          Type III     eps      df        MS         F     Sig.      et2_G     Obs.    SE     95% CI   lambda   Obs.  
                                     SS                                                                                                 Power 
cue          Sphericity Assumed   4.003e-05       -        2   2.001e-05   0.020   0.980   3.243e-04     19   0.008    0.015    0.021   0.051 
             Greenhouse-Geisser   4.003e-05   0.975    1.949   2.054e-05   0.020   0.979   3.243e-04     19   0.008    0.015    0.021   0.051 
             Huynh-Feldt          4.003e-05   0.975    1.949   2.054e-05   0.020   0.979   3.243e-04     19   0.008    0.015    0.021   0.051 
             Box                  4.003e-05   0.500        1   4.003e-05   0.020   0.890   

In [30]:
wpk_ffa=pd.read_csv("ffa_rho_df.csv", index_col='idx')

condlist = ['hc', 'nc', 'fc']
ffacues = wpk_ffa[condlist]
ffa_cues_long = ffacues.stack().reset_index(name='peak').rename(columns={'level_1':'cue'})
FFA_CUES = DF(ffa_cues_long)
FFA_CUE_RMA=Anova()
FFA_CUE_RMA.run(FFA_CUES, 'peak', wfactors=['cue'], sub='idx')

os.chdir('/Users/kyle/Dropbox/PFH/iPFH/PK_ANOVA/')
# Write output to a file
with open('FFA_CUE_RMA.txt','wb') as f:
        f.write("Face Regions, Cue Peak Comparisons:\n\n")
        f.write("Face Cue > House Cue:\nt(18) = %.4f, p=%.4f\n\n" % ttest_rel(wpk_ffa['fc'], wpk_ffa['hc']))
        f.write("Face Cue > Neutral Cue:\nt(18) = %.4f, p=%.4f\n\n\n\n" % ttest_rel(wpk_ffa['fc'], wpk_ffa['nc']))
        f.write("\n\n")
        f.write(str(FFA_CUE_RMA))


print "Face Regions, Cue Peak Comparisons"
print "Face Cue > House Cue: t(18) = %.4f, p=%.4f" % ttest_rel(wpk_ffa['fc'], wpk_ffa['hc'])
print "Face Cue > Neutral Cue: t(18) = %.4f, p=%.4f" % ttest_rel(wpk_ffa['fc'], wpk_ffa['nc'])

print(FFA_CUE_RMA)

Face Regions, Cue Peak Comparisons
Face Cue > House Cue: t(18) = 3.2614, p=0.0043
Face Cue > Neutral Cue: t(18) = 0.6390, p=0.5309
peak ~ cue

TESTS OF WITHIN SUBJECTS EFFECTS

Measure: peak
  Source                          Type III    eps      df      MS       F     Sig.    et2_G   Obs.    SE     95% CI   lambda   Obs.  
                                     SS                                                                                        Power 
cue          Sphericity Assumed      0.093       -        2   0.047   2.837   0.072   0.029     19   0.030    0.060    2.994   0.298 
             Greenhouse-Geisser      0.093   0.863    1.725   0.054   2.837   0.081   0.029     19   0.030    0.060    2.994   0.276 
             Huynh-Feldt             0.093   0.863    1.725   0.054   2.837   0.081   0.029     19   0.030    0.060    2.994   0.276 
             Box                     0.093   0.500        1   0.093   2.837   0.109   0.029     19   0.030    0.060    2.994   0.212 
-----

# RM ANOVA Including Region as a Factor

In [38]:

CSTIM_DF = DF(cstimdf.reset_index())
CStim_RMA=Anova()
CStim_RMA.run(CSTIM_DF, 'peak', wfactors=['cue', 'stim', 'region'], sub='idx')

os.chdir('/Users/kyle/Dropbox/PFH/iPFH/PK_ANOVA/')
# Write output to a file
with open('CUExSTIMxREGION_RMA.txt','wb') as f:
        f.write(str(CStim_RMA))

# Display output inline
print(CStim_RMA)

peak ~ cue * stim * region

TESTS OF WITHIN SUBJECTS EFFECTS

Measure: peak
   Source                            Type III    eps      df      MS       F        Sig.        et2_G     Obs.    SE     95% CI   lambda    Obs.  
                                        SS                                                                                                  Power 
cue             Sphericity Assumed      0.020       -        2   0.010    0.351       0.706       0.002     76   0.020    0.039     1.481   0.166 
                Greenhouse-Geisser      0.020   0.931    1.861   0.011    0.351       0.691       0.002     76   0.020    0.039     1.481   0.161 
                Huynh-Feldt             0.020   0.931    1.861   0.011    0.351       0.691       0.002     76   0.020    0.039     1.481   0.161 
                Box                     0.020   0.500        1   0.020    0.351       0.561       0.002     76   0.020    0.039     1.481   0.129 
------------------------------------------

# 1Way ANOVA Example (not appropriate for repeated measures!)

In [19]:
from pyvttbl import Anova1way
d = [[21.0, 20.0, 26.0, 46.0, 35.0, 13.0, 41.0, 30.0, 42.0, 26.0],
     [23.0, 30.0, 34.0, 51.0, 20.0, 38.0, 34.0, 44.0, 41.0, 35.0],
     [35.0, 35.0, 52.0, 29.0, 54.0, 32.0, 30.0, 42.0, 50.0, 21.0],
     [44.0, 40.0, 33.0, 45.0, 45.0, 30.0, 46.0, 34.0, 49.0, 44.0],
     [39.0, 44.0, 51.0, 47.0, 50.0, 45.0, 39.0, 51.0, 39.0, 55.0]]
conditions_list = 'Contact Hit Bump Collide Smash'.split()
D=Anova1way()
D.run(d, conditions_list=conditions_list)
print(D)

Anova: Single Factor on Measure

SUMMARY
Groups    Count   Sum   Average   Variance 
Contact      10   300        30    116.444 
Hit          10   350        35     86.444 
Bump         10   380        38    122.222 
Collide      10   410        41     41.556 
Smash        10   460        46     33.333 

O'BRIEN TEST FOR HOMOGENEITY OF VARIANCE
Source of Variation       SS       df      MS         F     P-value   eta^2   Obs. power 
Treatments             68081.975    4   17020.494   1.859     0.134   0.142        0.498 
Error                 412050.224   45    9156.672                                        
Total                 480132.199   49                                                    

ANOVA
Source of Variation    SS    df   MS      F     P-value   eta^2   Obs. power 
Treatments            1460    4   365   4.562     0.004   0.289        0.837 
Error                 3600   45    80                                        
Total                 5060   49                     

In [2]:
#ewma = pd.read_csv("HDDM/Final/allsx_ewma.csv")
#allsx = pd.read_csv("fMRI/behav/all_sx.csv")
#allcor = pd.read_csv("fMRI/behav/all_sx_cor.csv")

In [3]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/')
diff = pd.read_csv('diff_rho_df.csv')

## WIN, BW, & WSS Peaks for FFA and PPA Regions

In [1]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/idxPK')
#WIN Peaks
wpk_ppa=pd.read_csv("ppa_rho_df.csv", index_col='idx')
wpk_ffa=pd.read_csv("ffa_rho_df.csv", index_col='idx')

#BW Peaks
bpk_ppa=pd.read_csv("ppa_bw_rhodf.csv", index_col='idx')
bpk_ffa=pd.read_csv("ffa_bw_rhodf.csv", index_col='idx')

#WSS Peaks
spk_ppa=pd.read_csv("ppa_wss_rhodf.csv", index_col='idx')
spk_ffa=pd.read_csv("ffa_wss_rhodf.csv", index_col='idx')

In [5]:
df = pd.read_csv("../idxPARAM/msms_rho_df.csv")

In [47]:
pearsonr(df['vHF_n_zH2b'], ssl_ppa['hcface'].values/bpk_ffa['hc-hcF'])

(0.065182192320241777, 0.79092061115947487)

In [15]:
pearsonr(df['vHF_n_zH2a'], (wpk_ppa['hcF']/wpk_ppa['hc-hcF']))

(-0.37715493941157546, 0.1114272172897016)

In [7]:
wtc_ffa['peak'] = wtc_ffa.loc[:, '1':'12'].max(axis=1)
wtc_ppa['peak'] = wtc_ppa.loc[:, '1':'12'].max(axis=1)

NameError: name 'wtc_ffa' is not defined

In [30]:
#wtc_ffa[wtc_ffa['cond']=='hcF'.groupby(['sx','cond'])

fpks = wtc_ffa.pivot(index='sx', columns='cond', values='peak')
hpks = wtc_ppa.pivot(index='sx', columns='cond', values='peak')

ttest_rel(fpks.hcface - fpks.fcface, hpks.hcface - hpks.fcface)

(1.8594074691163476, 0.079394923721580968)

In [9]:
ttest_rel(wpk_ppa.hcH-wpk_ppa.fcH, wpk_ffa.fcF-wpk_ffa.hcF)
x1 = (wpk_ffa.fcF-wpk_ffa.hcF)/np.sqrt(stats.sem(wpk_ffa.fcF-wpk_ffa.hcF))
x2 = (wpk_ppa.hcH-wpk_ppa.fcH)/np.sqrt(stats.sem(wpk_ppa.hcH-wpk_ppa.fcH))
ttest_rel(x1, x2)

(-0.071013173522570583, 0.94417032731039185)

In [17]:
(wpk_ppa.hcH-wpk_ppa.fcH).mean()
(wpk_ffa.fcF-wpk_ffa.hcF).mean()

-0.081192272105263155

## WIN & WSS Min for FFA and PPA Regions

In [10]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/idxLO')

#WIN Minimum of TC
wlo_ppa=pd.read_csv("win_ppalow_rhodf.csv", index_col='idx')
wlo_ffa=pd.read_csv("win_ffalow_rhodf.csv", index_col='idx')

#WSS Minimum of TC
slo_ppa=pd.read_csv("wss_ppalow_rhodf.csv", index_col='idx')
slo_ffa=pd.read_csv("wss_ffalow_rhodf.csv", index_col='idx')

## WIN, BW, & WSS timecourses for FFA and PPA Regions

In [14]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/idxTC')

#WIN Timecourses
wtc_ffa = pd.read_csv('WIN_AllSx_FaceClusters.csv')
wtc_ppa = pd.read_csv('WIN_AllSx_HouseClusters.csv')

#BW Timecourses
btc_ffa = pd.read_csv('BW_AllSx_FaceClusters.csv')
btc_ppa = pd.read_csv('BW_AllSx_HouseClusters.csv')

#WSS Timecourses
stc_ffa = pd.read_csv('WSS_AllSx_FaceClusters.csv')
stc_ppa = pd.read_csv('WSS_AllSx_HouseClusters.csv')

In [37]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/idxLO')

ppa_lo = wtc_ppa.loc[:, '1':'12'].min(axis=1)
xdf = pd.DataFrame({"idx":wtc_ppa.idx, "cond":wtc_ppa.cond, 'low':ppa_lo})
wlo_ppa['fc'] = xdf[xdf.cond=='fc'].low.values
wlo_ppa['nc'] = xdf[xdf.cond=='nc'].low.values
wlo_ppa['hc'] = xdf[xdf.cond=='hc'].low.values
wlo_ppa['fcF'] = xdf[xdf.cond=='fcface'].low.values
wlo_ppa['fcH'] = xdf[xdf.cond=='fchouse'].low.values
wlo_ppa['ncF'] = xdf[xdf.cond=='ncface'].low.values
wlo_ppa['ncH'] = xdf[xdf.cond=='nchouse'].low.values
wlo_ppa['hcF'] = xdf[xdf.cond=='hcface'].low.values
wlo_ppa['hcH'] = xdf[xdf.cond=='hchouse'].low.values
wlo_ppa.to_csv('win_ppalow_rhodf.csv')

# FFA MIN
ffa_lo = wtc_ffa.loc[:, '1':'12'].min(axis=1)
xdf = pd.DataFrame({"idx":wtc_ffa.idx, "cond":wtc_ffa.cond, 'low':ffa_lo})

wlo_ffa['fc'] = xdf[xdf.cond=='fc'].low.values
wlo_ffa['nc'] = xdf[xdf.cond=='nc'].low.values
wlo_ffa['hc'] = xdf[xdf.cond=='hc'].low.values
wlo_ffa['fcF'] = xdf[xdf.cond=='fcface'].low.values
wlo_ffa['fcH'] = xdf[xdf.cond=='fchouse'].low.values
wlo_ffa['ncF'] = xdf[xdf.cond=='ncface'].low.values
wlo_ffa['ncH'] = xdf[xdf.cond=='nchouse'].low.values
wlo_ffa['hcF'] = xdf[xdf.cond=='hcface'].low.values
wlo_ffa['hcH'] = xdf[xdf.cond=='hchouse'].low.values

wlo_ffa.to_csv('win_ffalow_rhodf.csv')


In [40]:
os.chdir('/Users/kyle/Desktop/')
acc = pd.read_csv("avg_acc.csv", index_col=0)
rt = pd.read_csv("avg_rt.csv", index_col=0)

In [43]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/idxPARAM/')
msm = pd.read_csv('msm_rho_df.csv')
msm.zN.mean()

0.6638945263157893

In [41]:
ttest_rel(acc.nHouse, acc.nFace)

(2.8756424349924461, 0.010060509923226248)

In [36]:
ttest_rel(wlo_ppa.fcF, wlo_ppa.hcF)

(4.7395059039074621, 0.0001636641695634319)

# WIN Slopes 

In [261]:
wtcf = wtc_ffa[~wtc_ffa.cond.isin(['fc', 'hc', 'nc'])]
wtch = wtc_ppa[~wtc_ppa.cond.isin(['fc', 'hc', 'nc'])]
ffa_win_slopes = pd.DataFrame({c:get_slopes(wtcf[wtcf.cond==c], start='2') for c in wtcf.cond.unique()}, index=wtcf.sx.unique())
ffa_win_slopes.index.name = 'idx'
ffa_win_slopes.to_csv("../IdxSL/ffa_win_slopes.csv")

ppa_win_slopes = pd.DataFrame({c:get_slopes(wtch[wtch.cond==c], start='2') for c in wtch.cond.unique()}, index=wtch.sx.unique())
ppa_win_slopes.index.name = 'idx'
ppa_win_slopes.to_csv("../IdxSL/ppa_win_slopes.csv")

# WIN Slopes To Long Format

In [262]:
ffawin_long = ffa_win_slopes.stack().reset_index(name='slope')
ffawin_long.rename(columns={'level_1':'cstim'}, inplace=True)
ppawin_long = ppa_win_slopes.stack().reset_index(name='slope')
ppawin_long.rename(columns={'level_1':'cstim'}, inplace=True)

ffawin_long['cue'] = ['fc', 'fc', 'hc', 'hc', 'nc', 'nc']*int(len(ffawin_long)/6)
ppawin_long['cue'] = ['fc', 'fc', 'hc', 'hc', 'nc', 'nc']*int(len(ppawin_long)/6)
ffawin_long['stim'] = ['face','house']*int(len(ffawin_long)/2)
ppawin_long['stim'] = ['face','house']*int(len(ppawin_long)/2)

ffawin_long.to_csv('../IdxSL/ffa_win_slope_long.csv', index=False)
ppawin_long.to_csv('../IdxSL/ppa_win_slope_long.csv', index=False)

# WIN Slopes (Neg)

In [185]:
wtcf = wtc_ffa[~wtc_ffa.cond.isin(['fc', 'hc', 'nc'])]
wtch = wtc_ppa[~wtc_ppa.cond.isin(['fc', 'hc', 'nc'])]

ffa_win_slopes = pd.DataFrame({c:get_slopes(wtcf[wtcf.cond==c], start='3', direction='neg') for c in wtcf.cond.unique()}, index=wtcf.sx.unique())
ffa_win_slopes.index.name = 'idx'
ffa_win_slopes.to_csv("../IdxSL/ffa_win_neg_slopes.csv")

ppa_win_slopes = pd.DataFrame({c:get_slopes(wtch[wtch.cond==c], start='3', direction='neg') for c in wtch.cond.unique()}, index=wtch.sx.unique())
ppa_win_slopes.index.name = 'idx'
ppa_win_slopes.to_csv("../IdxSL/ppa_win_neg_slopes.csv")

# WIN Slopes (Neg) To Long Format

In [186]:
ffawin_long = ffa_win_slopes.stack().reset_index(name='slope')
ffawin_long.rename(columns={'level_1':'cstim'}, inplace=True)
ppawin_long = ppa_win_slopes.stack().reset_index(name='slope')
ppawin_long.rename(columns={'level_1':'cstim'}, inplace=True)

ffawin_long['cue'] = ['fc', 'fc', 'hc', 'hc', 'nc', 'nc']*int(len(ffawin_long)/6)
ppawin_long['cue'] = ['fc', 'fc', 'hc', 'hc', 'nc', 'nc']*int(len(ppawin_long)/6)
ffawin_long['stim'] = ['face','house']*int(len(ffawin_long)/2)
ppawin_long['stim'] = ['face','house']*int(len(ppawin_long)/2)

ffawin_long.to_csv('../IdxSL/ffa_win_neg_slope_long.csv', index=False)
ppawin_long.to_csv('../IdxSL/ppa_win_neg_slope_long.csv', index=False)

# WSS Slopes 

In [229]:
stch = stc_ppa[stc_ppa.cond.isin(['FFast', 'FSlow', 'HFast', 'HSlow'])]
stcf = stc_ffa[stc_ffa.cond.isin(['FFast', 'FSlow', 'HFast', 'HSlow'])]

ffa_wss_slopes = pd.DataFrame({c:get_slopes(stcf[stcf.cond==c], start='2') for c in stcf.cond.unique()}, index=stcf.sx.unique())
ffa_wss_slopes.index.name = 'idx'
ffa_wss_slopes.to_csv("../IdxSL/ffa_wss_slopes.csv")

ppa_wss_slopes = pd.DataFrame({c:get_slopes(stch[stch.cond==c], start='2') for c in stch.cond.unique()}, index=stch.sx.unique())
ppa_wss_slopes.index.name = 'idx'
ppa_wss_slopes.to_csv("../IdxSL/ppa_wss_slopes.csv")

# WSS Slopes To Long Format

In [230]:
ffawss_long = ffa_wss_slopes.stack().reset_index(name='slope')
ffawss_long.rename(columns={'level_1':'stimspd'}, inplace=True)
ppawss_long = ppa_wss_slopes.stack().reset_index(name='slope')
ppawss_long.rename(columns={'level_1':'stimspd'}, inplace=True)

ffawss_long['stim'] = ['face','face', 'house','house']*int(len(ffawss_long)/4)
ppawss_long['stim'] = ['face','face', 'house','house']*int(len(ppawss_long)/4)
ffawss_long['speed'] = ['fast','slow']*int(len(ffawss_long)/2)
ppawss_long['speed'] = ['fast','slow']*int(len(ppawss_long)/2)

ppawss_long.to_csv('../IdxSL/ppa_wss_slope_long.csv', index=False)
ffawss_long.to_csv('../IdxSL/ffa_wss_slope_long.csv', index=False)

# WSS Slopes (Neg)

In [189]:
stch = stc_ppa[stc_ppa.cond.isin(['FFast', 'FSlow', 'HFast', 'HSlow'])]
stcf = stc_ffa[stc_ffa.cond.isin(['FFast', 'FSlow', 'HFast', 'HSlow'])]

ffa_wss_slopes = pd.DataFrame({c:get_slopes(stcf[stcf.cond==c], start='3', direction='neg') for c in stcf.cond.unique()}, index=stcf.sx.unique())
ffa_wss_slopes.index.name = 'idx'
ffa_wss_slopes.to_csv("../IdxSL/ffa_wss_neg_slopes.csv")

ppa_wss_slopes = pd.DataFrame({c:get_slopes(stch[stch.cond==c], start='3', direction='neg') for c in stch.cond.unique()}, index=stch.sx.unique())
ppa_wss_slopes.index.name = 'idx'
ppa_wss_slopes.to_csv("../IdxSL/ppa_wss_neg_slopes.csv")

# WSS Slopes To Long Format (Neg)

In [190]:
ffawss_long = ffa_wss_slopes.stack().reset_index(name='slope')
ffawss_long.rename(columns={'level_1':'stimspd'}, inplace=True)
ppawss_long = ppa_wss_slopes.stack().reset_index(name='slope')
ppawss_long.rename(columns={'level_1':'stimspd'}, inplace=True)

ffawss_long['stim'] = ['face','face', 'house','house']*int(len(ffawss_long)/4)
ppawss_long['stim'] = ['face','face', 'house','house']*int(len(ppawss_long)/4)
ffawss_long['speed'] = ['fast','slow']*int(len(ffawss_long)/2)
ppawss_long['speed'] = ['fast','slow']*int(len(ppawss_long)/2)

ppawss_long.to_csv('../IdxSL/ppa_wss_neg_slope_long.csv', index=False)
ffawss_long.to_csv('../IdxSL/ffa_wss_neg_slope_long.csv', index=False)

# PPA CStim Slope RMA

In [263]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/idxSL/')
wsl_ppa=pd.read_csv("ppa_win_slope_long.csv")
fch_sl = wsl_ppa[wsl_ppa.cstim=='fchouse'].slope
hch_sl = wsl_ppa[wsl_ppa.cstim=='hchouse'].slope
nch_sl = wsl_ppa[wsl_ppa.cstim=='nchouse'].slope
ttrel(fch_sl, hch_sl)

(2.7890620934005987, 0.012117300811849441)

In [264]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/idxSL/')
wsl_ppa=pd.read_csv("ppa_win_slope_long.csv")

PPA_CStim_Slp = DF(wsl_ppa)
PPA_CStimSLP_RMA=Anova()
PPA_CStimSLP_RMA.run(PPA_CStim_Slp, 'slope', wfactors=['cue', 'stim'], sub='idx')

os.chdir('/Users/kyle/Dropbox/PFH/iPFH/SL_ANOVA/')
# Write output to a file
with open('PPA_CStimSLP_RMA.txt','wb') as f:
      f.write("FC House Slope > HC House Slope (Neg):\nt(18) = %.4f, p=%.4f\n\n" % ttrel(fch_sl, hch_sl))
      f.write("so a faster deflection for unexpected houses than expected houses in house-selective regions")
      f.write("NC House Slope > HC House Slope (Neg):\nt(18) = %.4f, p=%.4f\n\n" % ttrel(nch_sl, hch_sl))
      f.write("so a faster deflection for neutral houses than expected houses in house-selective regions")
      f.write(str(PPA_CStimSLP_RMA))

# Display output inline
print(PPA_CStimSLP_RMA)

slope ~ cue * stim

TESTS OF WITHIN SUBJECTS EFFECTS

Measure: slope
  Source                           Type III    eps      df        MS         F        Sig.      et2_G   Obs.    SE     95% CI   lambda    Obs.  
                                      SS                                                                                                  Power 
cue           Sphericity Assumed      0.019       -        2       0.009    4.818       0.014   0.062     38   0.007    0.015    10.172   0.787 
              Greenhouse-Geisser      0.019   0.952    1.904       0.010    4.818       0.016   0.062     38   0.007    0.015    10.172   0.772 
              Huynh-Feldt             0.019   0.952    1.904       0.010    4.818       0.016   0.062     38   0.007    0.015    10.172   0.772 
              Box                     0.019   0.500        1       0.019    4.818       0.042   0.062     38   0.007    0.015    10.172   0.569 
-------------------------------------------------------------

# PPA CStim Neg Slope RMA

In [195]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/idxSL/')
wsl_ppa=pd.read_csv("ppa_win_neg_slope_long.csv")

fcf_sl = wsl_ppa[wsl_ppa.cstim=='fcface'].slope
hcf_sl = wsl_ppa[wsl_ppa.cstim=='hcface'].slope
ncf_sl = wsl_ppa[wsl_ppa.cstim=='ncface'].slope

PPA_CStim_NegSlp = DF(wsl_ppa)
PPA_CStim_NegSLP_RMA=Anova()
PPA_CStim_NegSLP_RMA.run(PPA_CStim_NegSlp, 'slope', wfactors=['cue', 'stim'], sub='idx')

os.chdir('/Users/kyle/Dropbox/PFH/iPFH/SL_ANOVA/')
# Write output to a file
with open('PPA_CStim_NegSLP_RMA.txt','wb') as f:
      f.write("House Regions, cStim Negative Slope Comparisons:\n\n")
      f.write("HC Face Slope (Neg) < FC Face Slope (Neg):\nt(18) = %.4f, p=%.4f\n\n" % ttrel(hcf_sl, fcf_sl))
      f.write("so a faster negative deflection for unexpected faces than expected faces in house-selective regions")
      f.write("NC Face Slope (Neg) < FC Face Slope (Neg):\nt(18) = %.4f, p=%.4f\n\n" % ttrel(hcf_sl, ncf_sl))
      f.write("so a faster negative deflection for neutral faces than expected faces in house-selective regions")

      f.write("\n\n")
      f.write(str(PPA_CStim_NegSLP_RMA))

# Display output inline
print(PPA_CStim_NegSLP_RMA)

slope ~ cue * stim

TESTS OF WITHIN SUBJECTS EFFECTS

Measure: slope
  Source                           Type III    eps      df        MS         F        Sig.      et2_G   Obs.    SE     95% CI   lambda   Obs.  
                                      SS                                                                                                 Power 
cue           Sphericity Assumed      0.004       -        2       0.002    3.567       0.039   0.037     38   0.004    0.007    7.531   0.650 
              Greenhouse-Geisser      0.004   0.902    1.803       0.002    3.567       0.044   0.037     38   0.004    0.007    7.531   0.617 
              Huynh-Feldt             0.004   0.902    1.803       0.002    3.567       0.044   0.037     38   0.004    0.007    7.531   0.617 
              Box                     0.004   0.500        1       0.004    3.567       0.075   0.037     38   0.004    0.007    7.531   0.451 
-------------------------------------------------------------------

# FFA CStim Slope RMA

In [267]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/idxSL/')
wsl_ffa=pd.read_csv("ffa_win_slope_long.csv")
fcf_sl = wsl_ffa[wsl_ffa.cstim=='fcface'].slope
hcf_sl = wsl_ffa[wsl_ffa.cstim=='hcface'].slope
ncf_sl = wsl_ffa[wsl_ffa.cstim=='ncface'].slope
ttrel(hcf_sl, fcf_sl)
#ttrel(fcf_sl, fcf_sl)

(2.0244733102702965, 0.058019215395044912)

In [268]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/idxSL/')
wsl_ffa=pd.read_csv("ffa_win_slope_long.csv")

FFA_CStim_Slp = DF(wsl_ffa)
FFA_CStimSLP_RMA=Anova()
FFA_CStimSLP_RMA.run(FFA_CStim_Slp, 'slope', wfactors=['cue', 'stim'], sub='idx')

os.chdir('/Users/kyle/Dropbox/PFH/iPFH/SL_ANOVA/')
# Write output to a file
with open('FFA_CStimSLP_RMA.txt','wb') as f:
      f.write("Face Regions, cStim Slope Comparisons:\n\n")
      f.write("HC Face Slope  > FC Face Slope (Neg):\nt(18) = %.4f, p=%.4f\n\n" % ttrel(hcf_sl, fcf_sl))
      f.write("so a faster deflection for unexpected faces than expected faces in face-selective regions")
      f.write("\n\n")
      f.write(str(FFA_CStimSLP_RMA))

# Display output inline
print(FFA_CStimSLP_RMA)

slope ~ cue * stim

TESTS OF WITHIN SUBJECTS EFFECTS

Measure: slope
  Source                           Type III    eps      df      MS       F        Sig.      et2_G   Obs.    SE     95% CI   lambda    Obs.  
                                      SS                                                                                              Power 
cue           Sphericity Assumed      0.032       -        2   0.016    5.748       0.007   0.033     38   0.009    0.017    12.134   0.858 
              Greenhouse-Geisser      0.032   0.737    1.475   0.022    5.748       0.014   0.033     38   0.009    0.017    12.134   0.767 
              Huynh-Feldt             0.032   0.737    1.475   0.022    5.748       0.014   0.033     38   0.009    0.017    12.134   0.767 
              Box                     0.032   0.500        1   0.032    5.748       0.028   0.033     38   0.009    0.017    12.134   0.644 
-------------------------------------------------------------------------------------

# PPA WSS Slope RMA

In [270]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/idxSL/')
ssl_ppa=pd.read_csv("ppa_wss_slope_long.csv")
hfast_sl = ssl_ppa[ssl_ppa.stimspd=='HFast'].slope
hslow_sl = ssl_ppa[ssl_ppa.stimspd=='HSlow'].slope
ffast_sl = ssl_ppa[ssl_ppa.stimspd=='FFast'].slope
fslow_sl = ssl_ppa[ssl_ppa.stimspd=='FSlow'].slope

ttrel(hfast_sl, hslow_sl)
#ttrel(ffast_sl, fslow_sl)

(2.6020431984870833, 0.018021784152942196)

In [271]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/idxSL/')
ssl_ppa=pd.read_csv("ppa_wss_slope_long.csv")

PPA_StimSpeed_Slp = DF(ssl_ppa)
PPA_StimSpeedSLP_RMA=Anova()
PPA_StimSpeedSLP_RMA.run(PPA_StimSpeed_Slp, 'slope', wfactors=['stim', 'speed'], sub='idx')

os.chdir('/Users/kyle/Dropbox/PFH/iPFH/SL_ANOVA/')
# Write output to a file
with open('PPA_StimSpeedSLP_RMA.txt','wb') as f:
      f.write("House Regions, Stim Speed Slope Comparisons:\n\n")
      f.write("Fast House > Slow House Slope:\nt(18) = %.4f, p=%.4f\n\n" % ttrel(hfast_sl, hslow_sl))
      f.write("so a faster deflection for fast house responses than slow house responses in house-selective regions")
      f.write("no sig. diff. for faces (in the positive deflections for fast & slow face resp.) --> SEE NEG. DEFLECTION ANALYSIS")
      f.write(str(PPA_StimSpeedSLP_RMA))

# Display output inline
print(PPA_StimSpeedSLP_RMA)

slope ~ stim * speed

TESTS OF WITHIN SUBJECTS EFFECTS

Measure: slope
   Source                           Type III   eps   df      MS         F        Sig.      et2_G   Obs.    SE     95% CI   lambda    Obs.  
                                       SS                                                                                            Power 
stim           Sphericity Assumed      0.052     -    1       0.052   54.844   7.227e-07   0.462     38   0.005    0.011   115.781   1.000 
               Greenhouse-Geisser      0.052     1    1       0.052   54.844   7.227e-07   0.462     38   0.005    0.011   115.781   1.000 
               Huynh-Feldt             0.052     1    1       0.052   54.844   7.227e-07   0.462     38   0.005    0.011   115.781   1.000 
               Box                     0.052     1    1       0.052   54.844   7.227e-07   0.462     38   0.005    0.011   115.781   1.000 
-----------------------------------------------------------------------------------------

# PPA WSS Neg Slope RMA

In [272]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/idxSL/')
ssl_ppa=pd.read_csv("ppa_wss_neg_slope_long.csv")
ffast_sl = ssl_ppa[ssl_ppa.stimspd=='FFast'].slope
fslow_sl = ssl_ppa[ssl_ppa.stimspd=='FSlow'].slope
ttrel(ffast_sl, fslow_sl)

(-3.5649690605450854, 0.0022130614626700486)

In [273]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/idxSL/')
ssl_ppa=pd.read_csv("ppa_wss_neg_slope_long.csv")

PPA_StimSpeed_NegSlp = DF(ssl_ppa)
PPA_StimSpeed_NegSLP_RMA=Anova()
PPA_StimSpeed_NegSLP_RMA.run(PPA_StimSpeed_NegSlp, 'slope', wfactors=['stim', 'speed'], sub='idx')

os.chdir('/Users/kyle/Dropbox/PFH/iPFH/SL_ANOVA/')
# Write output to a file
with open('PPA_StimSpeed_NegSLP_RMA.txt','wb') as f:
      f.write("House Regions, Stim Speed Negative Slope Comparisons:\n\n")
      f.write("Fast Face > Slow Face Slope:\nt(18) = %.4f, p=%.4f\n\n" % ttrel(ffast_sl, fslow_sl))
      f.write("so a faster negative deflection for fast face responses than slow face responses in house-selective regions")
      f.write(str(PPA_StimSpeed_NegSLP_RMA))

# Display output inline
print(PPA_StimSpeed_NegSLP_RMA)

slope ~ stim * speed

TESTS OF WITHIN SUBJECTS EFFECTS

Measure: slope
   Source                           Type III   eps   df      MS         F        Sig.      et2_G   Obs.    SE     95% CI   lambda   Obs.  
                                       SS                                                                                           Power 
stim           Sphericity Assumed      0.019     -    1       0.019   20.343   2.708e-04   0.363     38   0.005    0.010   42.946   1.000 
               Greenhouse-Geisser      0.019     1    1       0.019   20.343   2.708e-04   0.363     38   0.005    0.010   42.946   1.000 
               Huynh-Feldt             0.019     1    1       0.019   20.343   2.708e-04   0.363     38   0.005    0.010   42.946   1.000 
               Box                     0.019     1    1       0.019   20.343   2.708e-04   0.363     38   0.005    0.010   42.946   1.000 
-----------------------------------------------------------------------------------------------

# FFA WSS Slope RMA

In [274]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/idxSL/')
ssl_ffa=pd.read_csv("ffa_wss_slope_long.csv")
ffast_sl = ssl_ffa[ssl_ffa.stimspd=='FFast'].slope
fslow_sl = ssl_ffa[ssl_ffa.stimspd=='FSlow'].slope

In [275]:
ttrel(ffast_sl, fslow_sl)

(7.5854414997553778, 5.1847147439727059e-07)

In [276]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/idxSL/')
ssl_ffa=pd.read_csv("ffa_wss_slope_long.csv")

FFA_StimSpeed_Slp = DF(ssl_ffa)
FFA_StimSpeedSLP_RMA=Anova()
FFA_StimSpeedSLP_RMA.run(FFA_StimSpeed_Slp, 'slope', wfactors=['stim', 'speed'], sub='idx')

os.chdir('/Users/kyle/Dropbox/PFH/iPFH/SL_ANOVA/')
# Write output to a file
with open('FFA_StimSpeedSLP_RMA.txt','wb') as f:
      f.write("Face Regions, Stim Speed Slope Comparisons:\n\n")
      f.write("Fast Face > Slow Face Slope:\nt(18) = %.4f, p=%.4f\n\n" % ttrel(ffast_sl, fslow_sl))
      f.write("so a faster deflection for fast face responses than slow face responses in face-selective regions")
      f.write("no diff. for houses")
      f.write(str(FFA_StimSpeedSLP_RMA))

# Display output inline
print(FFA_StimSpeedSLP_RMA)

slope ~ stim * speed

TESTS OF WITHIN SUBJECTS EFFECTS

Measure: slope
   Source                           Type III   eps   df    MS        F        Sig.      et2_G   Obs.    SE     95% CI   lambda    Obs.  
                                       SS                                                                                         Power 
stim           Sphericity Assumed      0.258     -    1   0.258   164.395   1.725e-10   1.072     38   0.007    0.014   347.056       1 
               Greenhouse-Geisser      0.258     1    1   0.258   164.395   1.725e-10   1.072     38   0.007    0.014   347.056       1 
               Huynh-Feldt             0.258     1    1   0.258   164.395   1.725e-10   1.072     38   0.007    0.014   347.056       1 
               Box                     0.258     1    1   0.258   164.395   1.725e-10   1.072     38   0.007    0.014   347.056       1 
-----------------------------------------------------------------------------------------------------------

In [453]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/idxPARAM/')
msm = pd.read_csv("msm_rho_df.csv")

In [16]:
ssl_ppa=pd.read_csv('/Users/kyle/Dropbox/PFH/iPFH/idxSL/ppa_win_neg_slopes.csv', index_col='idx')
ssl_ffa=pd.read_csv('/Users/kyle/Dropbox/PFH/iPFH/idxSL/ffa_win_slopes.csv', index_col='idx')
#ssl_ppa_neg=pd.read_csv('/Users/kyle/Dropbox/PFH/iPFH/idxSL/ppa_wss_neg_slopes.csv', index_col='idx')

In [455]:
x = ssl_ffa+abs(ssl_ppa)

In [19]:
ssl_ffa['fcface'].values/wpk_ffa['fc-fcF']

idx
8     1.9244
10    0.9646
11    0.8780
13    1.6781
15    0.3508
16   -0.4930
17    0.3950
18    0.4049
19    0.6424
20    0.4827
21    0.3493
22    0.4692
23    0.3459
24    2.1940
26    0.6751
27   -1.1522
29    0.5173
30    1.8641
31    0.9098
Name: fc-fcF, dtype: float64

In [424]:
spearmanr(x['FFast'], ssm['vFFast_n_z2a'])

(-0.21403508771929824, 0.37892417299512104)

In [486]:
pearsonr(ssl_ffa['hcface'], msm['vHF'])

(-0.26716501537889864, 0.26884010321853258)

In [387]:
ssm['vFFast_n_z2a'] = ssm['vFFast']/ssm['z2a']
ssm['vFSlow_n_z2a'] = ssm['vFSlow']/ssm['z2a']

In [290]:
ssm

,a,vFFast,vHFast,vFSlow,vHSlow,z,zF,zH,z2a,z2b,vFFast_n_z2a,vHFast_n_z2b,vFSlow_n_z2a,vHSlow_n_z2b
0,4.1789,6.7562,7.4927,1.2652,1.8542,0.5881,0.5881,0.6067,1.7214,2.4575,3.9247,3.0490,0.7349,0.7545
1,3.7056,1.4063,2.4281,0.2866,0.8312,0.5889,0.5889,0.6384,1.5235,2.1821,0.9231,1.1127,0.1881,0.3809
2,3.1099,0.7858,0.6107,-0.0054,0.2850,0.5990,0.5990,0.6076,1.2470,1.8629,0.6302,0.3278,-0.0043,0.1530
3,5.4528,4.9719,5.8098,1.2016,1.7759,0.6596,0.6596,0.6030,1.8561,3.5967,2.6787,1.6153,0.6474,0.4938
4,2.8284,4.2809,4.3525,0.3571,1.2017,0.6901,0.6901,0.5792,0.8766,1.9518,4.8836,2.2300,0.4073,0.6157
5,2.6929,3.8178,4.1451,0.2581,1.2953,0.6408,0.6408,0.6704,0.9674,1.7255,3.9465,2.4023,0.2668,0.7507
6,3.2235,1.2634,1.1130,0.1205,0.4427,0.6419,0.6419,0.5382,1.1544,2.0692,1.0945,0.5379,0.1043,0.2140
7,3.9309,2.0138,2.7896,0.0986,1.0736,0.6693,0.6693,0.6395,1.2998,2.6311,1.5493,1.0602,0.0759,0.4080
8,3.5984,1.9997,2.5636,0.1411,1.0321,0.6517,0.6517,0.6643,1.2532,2.3452,1.5957,1.0931,0.1126,0.4401
9,4.1240,4.1403,4.6462,0.5654,0.7241,0.6267,0.6267,0.5952,1.5397,2.5844,2.6891,1.7978,0.3672,0.2802


# Transform WIN cSTIM DF from Wide to Long for RM-ANOVA

In [31]:
ffa_cstim = wpk_ffa[['hcF', 'ncF', 'fcF', 'hcH', 'ncH', 'fcH']]
ppa_cstim = wpk_ppa[['hcF', 'ncF', 'fcF', 'hcH', 'ncH', 'fcH']]

dflist=[]
for cond, cdata in ffa_cstim.iteritems():
      dflist.append(pd.DataFrame({'cue':[cond[:2]]*len(cdata), 'stim':[cond[-1]]*len(cdata), 'peak':cdata, 'region':'FFA'}))
for cond, cdata in ppa_cstim.iteritems():
      dflist.append(pd.DataFrame({'cue':[cond[:2]]*len(cdata), 'stim':[cond[-1]]*len(cdata), 'peak':cdata, 'region':'PPA'}))

cstimdf = pd.concat(dflist)

In [7]:
ffacue = wpk_ffa[['hc', 'nc', 'fc']]
ppacue = wpk_ppa[['hc', 'nc', 'fc']]
regn=['FFA', 'PPA']
dflist=[]
for i, df in enumerate([ffacue, ppacue]):
      for cond, cdata in df.iteritems():
            dflist.append(pd.DataFrame({'cue':[cond]*len(cdata),'peak':cdata,'region':regn[i]}))
cuesdf = pd.concat(dflist)

# Save RM-ANOVA DF's

In [71]:
#os.mkdir('/Users/kyle/Dropbox/PFH/iPFH/PK_ANOVA')
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/PK_ANOVA')
df.query('region=="FFA"').to_csv('WIN_FFA_cSTIM_StimPhase.csv')
df.query('region=="PPA"').to_csv('WIN_PPA_cSTIM_StimPhase.csv')

In [17]:
os.chdir('/Users/kyle/Dropbox/PFH/iPFH/PK_ANOVA')
cuesdf.query('region=="FFA"').to_csv('WIN_FFA_CuePhase.csv')
cuesdf.query('region=="PPA"').to_csv('WIN_PPA_CuePhase.csv')

# T-Test Valid, Invalid Slopes

In [54]:
ppay=[wtc_ppa.query('cond=="hchouse"'), wtc_ppa.query('cond=="fchouse"'), 
   wtc_ppa.query('cond=="fcface"'), wtc_ppa.query('cond=="hcface"')]

ffay=[wtc_ffa.query('cond=="hchouse"'), wtc_ffa.query('cond=="fchouse"'), 
   wtc_ffa.query('cond=="fcface"'), wtc_ffa.query('cond=="hcface"')]

for i, d in enumerate([ppay, ffay]):
      print ttest_rel(get_slopes(d[0]), get_slopes(d[1]))
      print ttest_rel(get_slopes(d[2]), get_slopes(d[3]))

(-2.7890620934029724, 0.012117300811787893)
(1.6427290427138803, 0.11779303065327808)
(0.98828696688222506, 0.3361117920044554)
(-2.024473310270904, 0.058019215394976772)


# T-Test Fast, Slow Slopes

In [9]:
ppay=[stc_ppa.query('cond=="HFast"'), stc_ppa.query('cond=="HSlow"'), 
   stc_ppa.query('cond=="FFast"'), stc_ppa.query('cond=="FSlow"')]
ffay=[stc_ffa.query('cond=="HFast"'), stc_ffa.query('cond=="HSlow"'), 
   stc_ffa.query('cond=="FFast"'), stc_ffa.query('cond=="FSlow"')]

for i, d in enumerate([ppay, ffay]):    
      print ttest_rel(get_slopes(d[0]), get_slopes(d[1]))
      print ttest_rel(get_slopes(d[2]), get_slopes(d[3]))

(2.6020431984881656, 0.018021784152901177)
(-0.25174767140368065, 0.80408796668406524)
(0.3719512218020577, 0.7142718380848434)
(7.5854414997653725, 5.1847147438776137e-07)


# T-Test Delta Bound

In [10]:
ppay=[wpk_ppa["hc-hcH"], wpk_ppa["fc-fcH"], 
    wpk_ppa["fc-fcF"], wpk_ppa["hc-hcF"]]

ffay=[wpk_ffa["hc-hcH"], wpk_ffa["fc-fcH"], 
    wpk_ffa["fc-fcF"],wpk_ffa["hc-hcF"]]

for i, d in enumerate([ppay, ffay]):    
      print ttest_rel(d[0], d[1])
      print ttest_rel(d[2], d[3])

(-2.7022402059562487, 0.014582115761102363)
(-1.0763078057320088, 0.29600461585608095)
(0.89884650297385538, 0.3806014947047166)
(-3.8823093691189894, 0.0010916141767539893)


# T-Test cStim Peaks

In [11]:
ppay=[wpk_ppa["hcH"], wpk_ppa["fcH"], 
    wpk_ppa["fcF"], wpk_ppa["hcF"]]

ffay=[wpk_ffa["hcH"], wpk_ffa["fcH"], 
    wpk_ffa["fcF"], wpk_ffa["hcF"]]

for i, d in enumerate([ppay, ffay]):    
      print ttest_rel(d[0], d[1])
      print ttest_rel(d[2], d[3])

(-3.1517337984922698, 0.0055173157118805771)
(-1.4262617459612432, 0.17090713143942196)
(-0.70091747359790535, 0.49231689341504747)
(-2.5256425319695994, 0.021148387293976834)


# T-Test Cue Peaks

In [13]:
ppay=[wpk_ppa["hc"], wpk_ppa["fc"], 
    wpk_ppa["fc"], wpk_ppa["hc"]]

ffay=[wpk_ffa["hc"], wpk_ffa["fc"], 
    wpk_ffa["fcF"], wpk_ffa["hcF"]]

for i, d in enumerate([ppay, ffay]):    
      print ttest_rel(d[0], d[1])
      print ttest_rel(d[2], d[3])

(-0.041906928813351216, 0.96703425001621868)
(0.041906928813351216, 0.96703425001621868)
(-3.2613555340243878, 0.0043355547181970914)
(-2.5256425319695994, 0.021148387293976834)


In [45]:
def get_slopes(df, start='2', direction='pos'):

      slopes=[]

      if 'idx' in df.columns:
            sub_id_string = 'idx'
      elif 'sx' in df.columns:
            sub_id_string = 'sx'

      for i, (sx, sxvec) in enumerate(df.groupby(sub_id_string)):
            
            xhigh=np.where(sxvec.values==sxvec[start].values)[1][0]
            yhigh=sxvec[start].values[0]

            if direction=='pos':
                  xlow=np.where(sxvec.values==sxvec[start].values)[1][0]
                  ylow=sxvec[start].values[0]
                  xhigh=np.where(sxvec.values==np.max(sxvec.ix[:,start:].values[0][1:]))[1][0]
                  yhigh=np.max(sxvec.ix[:,start:].values)

            elif direction=='neg':
                  xlow=np.where(sxvec.values==np.min(sxvec.ix[:,start:].values[0][1:]))[1][0]
                  ylow=np.min(sxvec.ix[:,start:].values)
                  xhigh=np.where(sxvec.values==sxvec[start].values)[1][0]
                  yhigh=sxvec[start].values[0]
            else: 
                  print "direction must be string, 'pos' or 'neg'"
                  return []

            slopes.append((yhigh-ylow)/(xhigh-xlow))
      return np.array(slopes)    


def corrplots(x, y, brain, param, stim, cond, test='pearson', start='1', direct='pos', normalize=False, calc_slope=True):
      conditions = [' '.join(stimcond) for stimcond in itertools.product(stim, cond)]
      clist=[blues[-4], blues[-3], reds[-4], reds[-2]]
      f, (ax1, ax2) = plt.subplots(1,2, figsize=(9,6), sharey=True, sharex=True)

      for i, (xi, yi) in enumerate(zip(x, y)):

            if i <= 1:
                  ax=ax1
            else: 
                  ax=ax2

            if calc_slope:
                  yi = get_slopes(yi, start, direction=direct)
            if normalize:
                  xi = (xi - xi.mean())/xi.std()
                  yi = (yi - yi.mean())/yi.std()
                  
            plot_regression(xi, yi, robust=False, color=clist[::-1][i], ax=ax, cond=conditions[i], test=test)

      for i, ax in enumerate([ax1, ax2]):
            ax.legend(loc=2, fontsize=13)
            ax.set_xlabel(param, fontsize=18)
            #ax.set_title(stim[i])
      ax1.set_ylabel(brain, fontsize=18)
      plt.tight_layout()
      return f
    
def plot_regression(x, y, robust=True, color='Navy', cond=None, title=None, test='pearson', ax=None):
      if ax is None:
            f, ax = plt.subplots(1)

      if test == 'ci':
            cis = boot.ci((x, y), statfunction=stats.linregress, alpha=.05, method='bca')[:,2]
            legend = '$\/CI_{95%s}[%.2f, %.2f]$' % ('\%', cis[0], cis[1])
      elif test in ['pearson', 'spearman']:
      
            if test=='pearson':       
                  rho, pval = pearsonr(x, y)
                  rho_id =';\/r_{p}='
            elif test=='spearman':  
                  rho, pval = spearmanr(x, y) 
                  rho_id =';\/r_{s}='
                  if pval<=.05: 
                        sym = '\star'   
                  #elif pval<=.1: sym = '\dagger'
                  else: 
                        sym = 'n.s.'
                        legend =r'$%s%.2f%s$' % (rho_id, rho, sym)
      else:
            raise ValueError('test must be ci, pearson, or spearman')
      
      if cond:
            legend = cond + legend
      sns.regplot(x, y, robust=robust, fit_reg=True, line_kws={"color":color, 'lw':2, 'label':legend}, 
                scatter_kws={'color':color, 'alpha':.6}, ax=ax)
      sns.despine()
      plt.tight_layout()